# 4D change analysis of near-continuous LiDAR time series for applications in geomorphic monitoring

## Time series-based change analysis of surface dynamics at a sandy beach

In this practical, you will perform time series-based surface change analysis on a time series of permanent TLS point clouds of the sandy beach at Kijkduin for a timespan of around 6 months (<a href="#references">Vos et al., 2022</a>). 

The objective is to assess surface dynamics with three methods: time series clustering (following <a href="#references">Kuschnerus et al., 2021</a>), Principal Component Analysis (PCA; following Frantzen et al., 2023), and 4D objects-by-change (following <a href="#references">Anders et al., 2021</a>). Look into the related articles for comparison of possible surface dynamics of the use case and help for deciding on suitable parameters, etc. And then you might find interesting new results for other datasets and settings!

## Software and data
This exercise should be solved using Python with the [`py4dgeo`](https://github.com/3dgeo-heidelberg/py4dgeo) library. The workflow will be introduced throughout this practical. Also, make use of the software documentations!

Use [CloudCompare](../../../software/software_cloudcompare.md) or GIS Software (e.g., [QGIS](../../../software/software_qgis.md)) to check the data and visualize your results.

The dataset is a subsampled version of the original time series, using 12-hourly epochs of point clouds and spatial subsampling to 50 cm. In the data directory `kijkduin`, you find the prepared input point clouds and a core points point cloud, which is manually cleaned from noise.

## Loading data and calculation of surface changes

Prepare the analysis by compiling the list of files (epochs) and read the timestamps from the file names (format `YYMMDD_hhmmss`) into `datetime` objects. Use the point cloud files and timestamps to create a py4dgeo `SpatiotemporalAnalysis` object. For this you need to instantiate the M3C2 algorithm. You can use the point cloud file `170115_150816_aoi_50cm.laz` as core points. Explore the point cloud properties in CloudCompare: 

* Considering the available point density and surface characteristics, what would be a suitable cylinder radius for the distance calculation?
* What would be a suitable approach to derive the surface normals in this topography and expected types of surface changes?

Hint: In this flat topography and predominant provess of sand deposition and erosion, it can be suitable to orient the normals purely vertically. In this case, they do not need to be computed, and you can [customize the py4dgeo algorithm](https://py4dgeo.readthedocs.io/en/latest/customization.html#Changing-search-directions) accordingly.

Use the first point cloud in the time series (list of files) as reference epoch. You can assume a registration error of 1.9 cm for the M3C2 distance calculation (cf. <a href="#references">Vos et al., 2022</a>).

Explore the spatiotemporal change information by visualizing the changes at a selected epoch, and visualizing the time series at a selected location. 

First, we start by setting up the Python environment and data.

<span style="color:red">Specify the path to the input data directory (e.g. 'kijkduin') on your computer:</span>

In [1]:
# import required modules
import os
import numpy as np
import py4dgeo
from datetime import datetime

# specify the data path
data_path = 'path-to-data'

# check if the specified path exists
if not os.path.isdir(data_path):
    print(f'ERROR: {data_path} does not exist')
    print('Please specify the correct path to the data directory by replacing <path-to-data> above.')

# sub-directory containing the point clouds
pc_dir = os.path.join(data_path, 'pointclouds')

# list of point clouds (time series)
pc_list = os.listdir(pc_dir)
pc_list[:5] # print the first elements

ERROR: path-to-data does not exist
Please specify the correct path to the data directory by replacing <path-to-data> above.


FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'path-to-data\\pointclouds'

In [8]:
?py4dgeo.SpatiotemporalAnalysis

<span style="color:blue">In the list of point cloud files you can see that we have one laz file per epoch available. The file name contains the timestamp of the epoch, respectively, in format `YYMMDD_hhmmss`. To use this information for our analysis, we read the timestamp information from the file names into `datetime` objects.</span> 

In [ ]:
# read the timestamps from file names
timestamps = []
for f in pc_list:
    if not f.endswith('.laz'):
        continue

    # get the timestamp from the file name
    timestamp_str = '_'.join(f.split('.')[0].split('_')[1:]) # yields YYMMDD_hhmmss

    # convert string to datetime object
    timestamp = datetime.strptime(timestamp_str, '%y%m%d_%H%M%S')
    timestamps.append(timestamp)

timestamps[:5]

<span style="color:blue">Now we use the point cloud files and timestamp information to create a `SpatiotemporalAnalysis` object</span> 

In [ ]:
analysis = py4dgeo.SpatiotemporalAnalysis(f'{data_path}/kijkduin.zip', force=True)

<span style="color:blue">As reference epoch, we use the first epoch in our time series:</span> 

In [ ]:
# specify the reference epoch
reference_epoch_file = os.path.join(pc_dir, pc_list[0])

# read the reference epoch and set the timestamp
reference_epoch = py4dgeo.read_from_las(reference_epoch_file)
reference_epoch.timestamp = timestamps[0]

# set the reference epoch in the spatiotemporal analysis object
analysis.reference_epoch = reference_epoch

<span style="color:blue">For epochs to be added, we now configure the M3C2 algorithm to derive the change values. We would like to set the normals purely vertically, so we define a customized computation of cylinder `directions`:</span> 

In [ ]:
# Inherit from the M3C2 algorithm class to define a custom direction algorithm
class M3C2_Vertical(py4dgeo.M3C2):
    def directions(self):
        return np.array([0, 0, 1]) # vertical vector orientation

# specify corepoints, here all points of the reference epoch
analysis.corepoints = reference_epoch.cloud[::]

# specify M3C2 parameters for our custom algorithm class
analysis.m3c2 = M3C2_Vertical(cyl_radii=(1.0,), max_distance=10.0, registration_error = 0.019)

<span style="color:blue">Now we add all the other epochs with their timestamps:</span> 

In [ ]:
# create a list to collect epoch objects
epochs = []
for e, pc_file in enumerate(pc_list[1:]):
    epoch_file = os.path.join(pc_dir, pc_file)
    epoch = py4dgeo.read_from_las(epoch_file)
    epoch.timestamp = timestamps[e]
    epochs.append(epoch)

# add epoch objects to the spatiotemporal analysis object
analysis.add_epochs(*epochs)

# print the spatiotemporal analysis data for 3 corepoints and 5 epochs, respectively
print(f"Space-time distance array:\n{analysis.distances[:3,:5]}")
print(f"Uncertainties of M3C2 distance calculation:\n{analysis.uncertainties['lodetection'][:3, :5]}")
print(f"Timestamp deltas of analysis:\n{analysis.timedeltas[:5]}")

<span style="color:blue">We visualize the changes in the scene for a selected epoch, together with the time series of surface changes at a selected location.</span> 

<span style="color:red">Specify a selected epoch id and a selected location by core point id (i.e., replace the `0` in the variables). A suitable location can be selected in the core points cloud in CloudCompare.</span>

In [ ]:
cp_idx_sel = 0 # selected core point index (e.g. 15162)
epoch_idx_sel = 0 # selected epoch index (e.g. 28)

# import plotting module
import matplotlib.pyplot as plt

# allow interactive rotation in notebook
%matplotlib inline

# create the figure
fig=plt.figure(figsize=(15,5))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)

# get the corepoints
corepoints = analysis.corepoints.cloud

# get change values of last epoch for all corepoints
distances = analysis.distances
distances_epoch = [d[epoch_idx_sel] for d in distances] 

# get the time series of changes at a specific core point locations
coord_sel = analysis.corepoints.cloud[cp_idx_sel]
timeseries_sel = distances[cp_idx_sel]

# get the list of timestamps from the reference epoch timestamp and timedeltas
timestamps = [t + analysis.reference_epoch.timestamp for t in analysis.timedeltas]

# plot the scene
d = ax1.scatter(corepoints[:,0], corepoints[:,1], c=distances_epoch[:], cmap='seismic_r', vmin=-1.5, vmax=1.5, s=1, zorder=1) 
plt.colorbar(d, format=('%.2f'), label='Distance [m]', ax=ax1, pad=.15)

# add the location of the selected coordinate
ax1.scatter(coord_sel[0], coord_sel[1], facecolor='yellow', edgecolor='black', s=100, zorder=2, label='Selected location', marker='*')
ax1.legend()

# configure the plot layout
ax1.set_xlabel('X [m]')
ax1.set_ylabel('Y [m]')
ax1.set_aspect('equal')
ax1.set_title('Changes at %s' % (analysis.reference_epoch.timestamp+analysis.timedeltas[epoch_idx_sel]))

# plot the time series
ax2.scatter(timestamps, timeseries_sel, s=5, color='black', label='Raw')
ax2.plot(timestamps, timeseries_sel, color='blue', label='Smoothed')
ax2.set_xlabel('Date')

# add the epoch of the plotted scene
ax2.scatter(timestamps[epoch_idx_sel], timeseries_sel[epoch_idx_sel], facecolor='yellow', marker='*', edgecolor='black', s=100, color='red', label='Selected epoch')
ax2.legend()

# format the date labels
import matplotlib.dates as mdates
dtFmt = mdates.DateFormatter('%b-%d') 
plt.gca().xaxis.set_major_formatter(dtFmt) 

# configure the plot layout
ax2.set_ylabel('Distance [m]')
ax2.grid()
ax2.set_ylim(-0.3,1.6)
ax2.set_title('Time series at selected location')

plt.tight_layout()
plt.show()

## Temporal smoothing

You are dealing with a temporal subset of the original hourly time series. The effect of temporal measurement variability may therefore be less pronounced (compared to the assessment in, e.g., [Anders et al., 2019](#references)). Nonetheless, you may apply temporal smoothing to reduce the influence of noise on your change analysis using a rolling median averaging of one week. This will also fill possible gaps in your data, e.g., lower ranges during poor atmospheric conditions or no data due to water coverage during tides on the lower beach part. 

Visualize the raw and smoothed change values in the time series of your selected location.

<span style="color:blue">We apply a rolling median with a defined temporal window of 14 epochs (corresponding to one week of 12-hourly point clouds) using the `temporal_averaging()` function in py4dgeo.</span> 

In [ ]:
analysis.smoothed_distances = py4dgeo.temporal_averaging(analysis.distances, smoothing_window=14)

<span style="color:blue">Now we can compare the raw and smoothed time series at our selected location:</span> 

In [ ]:
# create the figure
fig, ax = plt.subplots(1,1,figsize=(7,5))

# plot the raw time series
ax.scatter(timestamps, timeseries_sel, color='blue', label='raw', s=5)

# plot the smoothed time series
timeseries_sel_smooth = analysis.smoothed_distances[cp_idx_sel]
ax.plot(timestamps, timeseries_sel_smooth, color='red', label='smooth')

# format the date labels
import matplotlib.dates as mdates
dtFmt = mdates.DateFormatter('%b-%d') 
plt.gca().xaxis.set_major_formatter(dtFmt) 

# add plot elements
ax.set_xlabel('Date')
ax.set_ylabel('Distance [m]')
ax.grid()
ax.set_ylim(-.25,1.25)

plt.tight_layout()
plt.show()

## Time series clustering
To derive characteristic change patterns on the sandy beach, perform k-means clustering of the time series following <a href="#references">Kuschnerus et al. (2021)</a>. Assess the clustering for different selections of `k` numbers of clusters.

* Can you interpret the characteristics of different parts on the beach? Visualize example time series for different clusters.
* From which number of clusters do you see a clear separation in overall units of the beach area?
* What are some detail change patterns that become visible for a higher number of clusters?


<span style="color:blue">We perform k-means clustering for a set of `k`s at once and collect the resulting labels for subsequent assessment:</span> 

In [ ]:
# import kmeans clustering module from scikit-learn
from sklearn.cluster import KMeans

# use the smoothed distances for clustering


# define the number of clusters


# perform clustering



<span style="color:blue">Now we can visualize the resulting change patterns:</span> 

<span style="color:blue"><span style="color:blue">We can look into the time series properties within selected clusters, to interpret the change pattern they are representing. For example, we can check the clusters by plotting the median values of all time series per cluster:</span> 

## Principal component analysis

In the following, we will use principal component analysis (PCA; also referred to as empirical orthogonal functions) as a tool to summarize the change patterns within a spatiotemporal dataset. The method is based on [this master thesis](http://resolver.tudelft.nl/uuid:ce98c4e3-6ca1-4966-a5cf-2120f2fa44bf), to be published in [Frantzen et al. (2023)](#references). 

For this technique, the 4D data shall be represented as a two-dimensional array in which the rows correspond to different epochs, and the columns correspond to each individual [x, y] coordinate where a z or surface change value is represented. 

Reshape the data from the `SpatiotemporalAnalysis` object so that all coordinates are represented in one axis, i.e. an array holding [time, space]:

In [ ]:
X = distances.transpose()

Find coordinates without nan values, and create a new array without nan values:

In [ ]:
idxs = np.argwhere(~np.isnan(X).any(axis=0)) # store the column indices where actual data is present
X_no_nan = X[:,idxs.flatten()].reshape((X.shape[0],-1))

Compute the loading and scores of the prinicpal components in 'NaN-free' X:

Plot the principal component loadings and scores along with their fraction of explained variance:

## Extraction of 4D objects-by-change

Now use the 4D objects-by-change (4D-OBC) method to identify individual surface activities in the beach scene. The objective is to extract temporary occurrences of accumulation or erosion, as occurs when a sand bar is formed during some timespan, or when sand is deposited by anthropogenic works. This type of surface activity is implemented with the original seed detection in py4dgeo, so you do not need to customize the algorithm. Decide for suitable parameters following <a href="#references">Anders et al. (2021)</a> - but bear in mind that we are using a different temporal resolution, so you may need to adjust the temporal change detection.

Perform the extraction for selected seed locations, e.g. considering interesting clusters of change patterns identified in the previous step. In principle, the spatiotemporal segmentation can also be applied to the full dataset (all time series at all core point locations are used as potential seed candidates), but long processing time needs to be expected.

<span style="color:blue">You may use a location to extract, e.g., sand bars as 4D object-by-change. The strength of the method is that the occurrences are identified individually, even though they have spatial overlap, as they can be separated in the time series information.</span> 

<span style="color:blue">We instantiate the `RegionGrowingAlgorithm` class using the parameters of [Anders et al, 2021](#references), and run the object extraction:</span> 

In [ ]:
# parametrize the 4D-OBC extraction

# run the algorithm


<span style="color:blue">We can first check the result by plotting the seeds in the time series:</span> 

<span style="color:blue">Next, we use the result and look into the spatial properties of one object (i.e. plotting the map of changes, for example, at the highest magnitude or middle epoch of the object duration, together with the seed time series):</span> 

<a id='references'></a>
# References

* Anders, K., Lindenbergh, R. C., Vos, S. E., Mara, H., de Vries, S., & Höfle, B. (2019). High-Frequency 3D Geomorphic Observation Using Hourly Terrestrial Laser Scanning Data Of A Sandy Beach. ISPRS Ann. Photogramm. Remote Sens. Spatial Inf. Sci., IV-2/W5, pp. 317-324. doi: [10.5194/isprs-annals-IV-2-W5-317-2019](https://doi.org/10.5194/isprs-annals-IV-2-W5-317-2019).
* Anders, K., Winiwarter, L., Mara, H., Lindenbergh, R., Vos, S. E., & Höfle, B. (2021). Fully automatic spatiotemporal segmentation of 3D LiDAR time series for the extraction of natural surface changes. ISPRS Journal of Photogrammetry and Remote Sensing, 173, pp. 297-308. doi: [10.1016/j.isprsjprs.2021.01.015](https://doi.org/10.1016/j.isprsjprs.2021.01.015).
* Frantzen, P., Voordendag, A., Kuschnerus, M., Rutzinger, M.,  Wouters, B., Lindenbergh, R. (2023, in review). Identifying Paraglacial Geomorphic Process Dynamics Using A Principal Component Analysis Of Terrestrial Laser Scanning Time Series.
* Kuschnerus, M., Lindenbergh, R., & Vos, S. (2021). Coastal change patterns from time series clustering of permanent laser scan data. Earth Surface Dynamics, 9 (1), pp. 89-103. doi: [10.5194/esurf-9-89-2021](https://doi.org/10.5194/esurf-9-89-2021).
* Vos, S., Anders, K., Kuschnerus, M., Lindenberg, R., Höfle, B., Aarnikhof, S. & Vries, S. (2022). A high-resolution 4D terrestrial laser scan dataset of the Kijkduin beach-dune system, The Netherlands.  Scientific Data, 9:191. doi: [10.1038/s41597-022-01291-9](https://doi.org/10.1038/s41597-022-01291-9).